<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Cost-Functions-and-Solutions-To-the-Optimization-Problem" data-toc-modified-id="Cost-Functions-and-Solutions-To-the-Optimization-Problem-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Cost Functions and Solutions To the Optimization Problem</a></span><ul class="toc-item"><li><span><a href="#The-Bad-News" data-toc-modified-id="The-Bad-News-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>The Bad News</a></span></li><li><span><a href="#The-Good-News" data-toc-modified-id="The-Good-News-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>The Good News</a></span></li></ul></li><li><span><a href="#Digging-Deeper-into-Logistic-Regression" data-toc-modified-id="Digging-Deeper-into-Logistic-Regression-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Digging Deeper into Logistic Regression</a></span><ul class="toc-item"><li><span><a href="#Preparing-Data" data-toc-modified-id="Preparing-Data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Preparing Data</a></span></li><li><span><a href="#Train-Logistic-Regression" data-toc-modified-id="Train-Logistic-Regression-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Train Logistic Regression</a></span></li><li><span><a href="#Interpreting-Logistic-Regression-Coefficients" data-toc-modified-id="Interpreting-Logistic-Regression-Coefficients-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Interpreting Logistic Regression Coefficients</a></span><ul class="toc-item"><li><span><a href="#Use-Coefficients-to-Generate-Prediction" data-toc-modified-id="Use-Coefficients-to-Generate-Prediction-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Use Coefficients to Generate Prediction</a></span></li></ul></li></ul></li><li><span><a href="#Exercise" data-toc-modified-id="Exercise-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Exercise</a></span></li><li><span><a href="#Level-Up" data-toc-modified-id="Level-Up-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Level Up</a></span><ul class="toc-item"><li><span><a href="#More-Generalizations:-Other-Link-Functions,-Other-Models" data-toc-modified-id="More-Generalizations:-Other-Link-Functions,-Other-Models-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>More Generalizations: Other Link Functions, Other Models</a></span></li></ul></li></ul></div>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# For our modeling steps
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import log_loss

# For demonstrative pruposes
from scipy.special import logit, expit

# Objectives

- Explain the form of logistic regression
- Explain how to interpret logistic regression coefficients
- Use logistic regression to perform a classification task

# Cost Functions and Solutions To the Optimization Problem

Unlike the least-squares problem for linear regression, no one has yet found a closed-form solution to the optimization problem presented by logistic regression. But even if one exists, the computation would no doubt be so complex that we'd be better off using some sort of approximation method instead.

But there's still a problem.

Recall the cost function for linear regression: <br/><br/>
$SSE = \Sigma_i(y_i - \hat{y}_i)^2 = \Sigma_i(y_i - (\beta_0 + \beta_1x_{i1} + ... + \beta_nx_{in}))^2$.

This function, $SSE(\vec{\beta})$, is convex.

If we plug in our new logistic equation for $\hat{y}$, we get: <br/><br/>
$SSE_{log} = \Sigma_i(y_i - \hat{y}_i)^2 = \Sigma_i\left(y_i - \left(\frac{1}{1+e^{-(\beta_0 + \beta_1x_{i1} + ... + \beta_nx_{in})}}\right)\right)^2$.

## The Bad News

*This* function, $SSE_{log}(\vec{\beta})$, is [**not** convex](https://towardsdatascience.com/why-not-mse-as-a-loss-function-for-logistic-regression-589816b5e03c).

That means that, if we tried to use gradient descent or some other approximation method that looks for the minimum of this function, we could easily find a local rather than a global minimum.

> Note that the scikit-learn class *expects the user to specify the solver* to be used in calculating the coefficients. The default solver, [lbfgs](https://en.wikipedia.org/wiki/Limited-memory_BFGS), works well for many applications.

## The Good News

We can use **log-loss** instead:

$\mathcal{L}(\vec{y}, \hat{\vec{y}}) = -\frac{1}{N}\Sigma^N_{i=1}\left(y_iln(\hat{y}_i)+(1-y_i)ln(1-\hat{y}_i)\right)$,

where $\hat{y}_i$ is the probability that $(x_{i1}, ... , x_{in})$ belongs to **class 1**.

**More resources on the log-loss function**:

https://towardsdatascience.com/optimization-loss-function-under-the-hood-part-ii-d20a239cde11

https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a

http://wiki.fast.ai/index.php/Log_Loss

# Digging Deeper into Logistic Regression

## Preparing Data

In [ ]:
# glass identification dataset
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/glass/glass.data'
col_names = ['id','ri','na','mg','al','si','k','ca','ba','fe','glass_type']
glass = pd.read_csv(url, names=col_names, index_col='id')
glass.sort_values('al', inplace=True)
glass.head()

In [ ]:
# types 1, 2, 3 are window glass
# types 5, 6, 7 are household glass
glass['household'] = glass.glass_type.map({1:0, 2:0, 3:0, 5:1, 6:1, 7:1})
glass.head()

## Train Logistic Regression

In [ ]:
# fit a logistic regression model and store the class predictions

logreg = LogisticRegression(random_state=42)
feature_cols = ['al']
X = glass[feature_cols]
y = glass.household
logreg.fit(X, y)
glass['household_pred_class'] = logreg.predict(X)

## Interpreting Logistic Regression Coefficients

In [ ]:
logreg.coef_

How do we interpret the coefficients of a logistic regression? For a linear regression, the situaton was like this:

- Linear Regression: We construct the best-fit line and get a set of coefficients. Suppose $\beta_1 = k$. In that case we would expect a 1-unit change in $x_1$ to produce a $k$-unit change in $y$.

- Logistic Regression: We find the coefficients of the best-fit line by some approximation method. Suppose $\beta_1 = k$. In that case we would expect a 1-unit change in $x_1$ to produce a $k$-unit change (not in $y$ but) in $ln\left(\frac{y}{1-y}\right)$.

We have:

$\ln\left(\frac{y(x_1+1, ... , x_n)}{1-y(x_1+1, ... , x_n)}\right) = \ln\left(\frac{y(x_1, ... , x_n)}{1-y(x_1, ... , x_n)}\right) + k$.

Exponentiating both sides:

$\frac{y(x_1+1, ... , x_n)}{1-y(x_1+1, ... , x_n)} = e^{\ln\left(\frac{y(x_1, ... , x_n)}{1-y(x_1, ... , x_n)}\right) + k}$ <br/><br/> $\frac{y(x_1+1, ... , x_n)}{1-y(x_1+1, ... , x_n)}= e^{\ln\left(\frac{y(x_1, ... , x_n)}{1-y(x_1, ... , x_n)}\right)}\cdot e^k$ <br/><br/> $\frac{y(x_1+1, ... , x_n)}{1-y(x_1+1, ... , x_n)}= e^k\cdot\frac{y(x_1, ... , x_n)}{1-y(x_1, ... , x_n)}$

That is, the odds ratio at $x_1+1$ has increased by a factor of $e^k$ relative to the odds ratio at $x_1$.

For more on interpretation, see [this page](https://support.minitab.com/en-us/minitab-express/1/help-and-how-to/modeling-statistics/regression/how-to/binary-logistic-regression/interpret-the-results/all-statistics-and-graphs/coefficients/).

In [ ]:
# examine the intercept

logreg.intercept_

**Interpretation:** For an 'al' value of 0, the log-odds of 'household' is -6.01.

In [ ]:
# convert log-odds to probability

logodds = logreg.intercept_
odds = np.exp(logodds)
prob = odds / (1 + odds)
prob

In [ ]:
# examine the coefficient for al

list(zip(feature_cols, logreg.coef_[0]))

**Interpretation:** A 1 unit increase in 'al' is associated with a 3.12 unit increase in the log-odds of 'household'.

Let's verify this as we change the aluminum content from 1 to 2.

In [ ]:
# Prediction for al=1

pred_al1 = logreg.predict_proba([[1]])
pred_al1

In [ ]:
# Odds ratio for al=1

odds_al1 = pred_al1[0][1] / pred_al1[0][0]

In [ ]:
# Prediction for al=2
pred_al2 = logreg.predict_proba([[2]])
pred_al2

In [ ]:
# Odds ratio for al=2

odds_al2 = pred_al2[0][1] / pred_al2[0][0]

In [ ]:
print((np.exp(logreg.coef_[0]) * odds_al1)[0])
print(odds_al2)

### Use Coefficients to Generate Prediction

In [ ]:
# compute predicted log-odds for al=2 using the equation

logodds = logreg.intercept_ + logreg.coef_[0] * 2
logodds

In [ ]:
# convert log-odds to odds

odds = np.exp(logodds)
odds

In [ ]:
# convert odds to probability

prob = odds / (1 + odds)
prob

In [ ]:
# compute predicted probability for al=2 using the predict_proba method

logreg.predict_proba(np.array([2]).reshape(1, 1))[:, 1]

# Exercise

Split the data below into train and test, and then convert the y-values (`geysers.kind`) into 1's and 0's. Then use `sklearn` to build a logistic regression model of whether Old Faithful's eruption wait time is long or short, based on the duration of the eruption. Finally, find the points in the test set where the model's prediction differs from the true y-value. How many are there?

In [ ]:
geysers = sns.load_dataset('geyser', **{'usecols': ['duration', 'kind']})

In [ ]:
geysers.head()

# Level Up

## More Generalizations: Other Link Functions, Other Models

Logistic regression's link function is the logit function, but different sorts of models use different link functions.

[Wikipedia](https://en.wikipedia.org/wiki/Generalized_linear_model#Link_function) has a nice table of generalized linear model types and their associated link functions.